# Flatiron Health mPC: Survival metrics for key elgibility criteria
**Background: Calculate survival metrics for emulated trials involving patients meeting key elgibliity criteria. Hazard ratio for the full cohort is calculated from a Cox-IPTW model. Restricted mean survival time and median overall survival are calculated for phenotypes using an IPTW-adjusted KM curve.** 

## Part 1: Preprocessing

### 1.1 Import packages and create necessary functions

In [1]:
import numpy as np
import pandas as pd

from scipy import stats

from sksurv.nonparametric import kaplan_meier_estimator
from survive import KaplanMeier, SurvivalData

from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.plotting import add_at_risk_counts
from lifelines.utils import median_survival_times, restricted_mean_survival_time
from lifelines.statistics import logrank_test

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

import warnings

In [2]:
# Function that returns number of rows and count of unique PatientIDs for a dataframe. 
def row_ID(dataframe):
    row = dataframe.shape[0]
    ID = dataframe['PatientID'].nunique()
    return row, ID

In [3]:
# Find index for value closest to input value. 
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [4]:
# Calculates median overeall suvival for risk groups. 
def mos(low, med, high, comp):
    low_os = low.median_survival_time_
    med_os = med.median_survival_time_
    high_os = high.median_survival_time_
    comp_os = comp.median_survival_time_
    mos = [low_os, med_os, high_os, comp_os]
    return (mos)

In [5]:
def rmst_mos_95ci(df, num_samples, drug, event, items_list, numerical_features, rmst_time):
    
    """
    Estimate the 95% confidence interval for RMST and mOS using bootstrap resampling.

    Parameters:
    - df: DataFrame containing survival data
    - num_samples: Number of bootstrap samples
    - drug: Treatment indicator variable
    - event: Event type ('death' or 'progression')
    - items_list: Feature list for IPTW 
    - numerical_features: List of numerical features
    - rmst_time: Time to calculate RMST 

    Returns:
    - mos_A_95: mOS 95% CI for treatment
    - mos_B_95: mOS 95% CI for control
    - rmst_A_95: RMST 95% CI for treatment
    - rmst_B_95: RMST 95% CI for control
    - difference_rmst_95: RMST 95% CI for difference between treatment and control 
    """
    
    np.random.seed(42)
    mos_A = []
    mos_B = []
    rmst_A_list = []
    rmst_B_list = []
    differences_rmst = []
    
    # Define variables based on the event type
    if event == 'death':
        time_column = 'timerisk_treatment'
        status_column = 'death_status'
        
    else:
        time_column = 'time_prog_treatment'
        status_column = 'pfs_status'
        
    # Set up preprocessor for logistical regression which will be for IPTW  
    numerical_transformer = Pipeline(steps = [
        ('imputer', SimpleImputer(strategy = 'median')),
        ('std_scaler', StandardScaler())])
        
    categorical_transformer = OneHotEncoder(handle_unknown = 'ignore')
    categorical_features = list(df.select_dtypes(include = ['category']).columns)
        
    preprocessor = ColumnTransformer(
        transformers = [
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)],
        remainder = 'passthrough')
    
    # Boostrap HR 
    for _ in range(num_samples):
        
        # Resample data with replacement
        resampled_df = resample(df).drop(columns = ['ps', 'weight'])
        
        # Calculated IPTW for the resampled group 
        df_x = preprocessor.fit_transform(resampled_df.filter(items = items_list))
                                           
        df_lr = LogisticRegression(max_iter = 1000)
        df_lr.fit(df_x, resampled_df[drug])
        warnings.filterwarnings("ignore")
                                           
        pred = df_lr.predict_proba(df_x)        
        resampled_df['ps'] = pred[:, 1]                          
        resampled_df['weight'] = (
                np.where(resampled_df[drug] == 1, 1/resampled_df['ps'], 1/(1 - resampled_df['ps'])))
    
        # mOS from IPTW-KM
        kmf_A = KaplanMeierFitter()
        kmf_A.fit(resampled_df.query(f'{drug} == 1')[time_column]/30,
                  resampled_df.query(f'{drug} == 1')[status_column], 
                  weights = resampled_df.query(f'{drug} == 1')['weight'])

        kmf_B = KaplanMeierFitter()
        kmf_B.fit(resampled_df.query(f'{drug} == 0')[time_column]/30,
                  resampled_df.query(f'{drug} == 0')[status_column], 
                  weights = resampled_df.query(f'{drug} == 0')['weight'])
    
        mos_A.append(kmf_A.median_survival_time_)
        mos_B.append(kmf_B.median_survival_time_)
        
        # RMST from IPTW-KM
        rmst_A = restricted_mean_survival_time(kmf_A, rmst_time)
        rmst_B = restricted_mean_survival_time(kmf_B, rmst_time)
        
        rmst_A_list.append(rmst_A)
        rmst_B_list.append(rmst_B)
        differences_rmst.append(rmst_A - rmst_B)

    # Calculate the 95% confidence interval
    results = pd.Series({
    'mos_A_95': np.percentile(mos_A, [2.5, 97.5]),
    'mos_B_95': np.percentile(mos_B, [2.5, 97.5]),
    'rmst_A_95': np.percentile(rmst_A_list, [2.5, 97.5]),
    'rmst_B_95': np.percentile(rmst_B_list, [2.5, 97.5]),
    'difference_rmst_95': np.percentile(differences_rmst, [2.5, 97.5])
    })
    
    return results

## Part 2: In silico trials 

### CHAARTED: docetaxel vs. ADT in metastatic, castration-sensitive prostate cancer  

**INCLUSION**
* Untreated metastatic prostate cancer, except up to 4 months of ADT 
* Castration-sensitive
* Received ADT or docetaxel plus ADT

#### ADT

In [6]:
df_full = pd.read_csv('df_risk_crude.csv', index_col = 'PatientID', dtype = {'death_status': bool})
df_full.index.nunique()

18927

In [7]:
df_full.reset_index(inplace = True)

In [8]:
adt = pd.read_csv('Enhanced_MetPC_ADT.csv')

In [9]:
adt = (
    adt[adt['PatientID'].isin(df_full['PatientID'])]
    .query('TreatmentSetting == "Advanced"')
)

In [10]:
row_ID(adt)

(17863, 17863)

In [11]:
adt.loc[:, 'StartDate'] = pd.to_datetime(adt['StartDate'])

In [12]:
adt = adt.rename(columns = {'StartDate': 'StartDate_adt'})

In [13]:
df_full = pd.merge(df_full, adt[['PatientID', 'StartDate_adt']], on = 'PatientID', how = 'left')

In [14]:
row_ID(df_full)

(18927, 18927)

In [15]:
enhanced_met = pd.read_csv('Enhanced_MetProstate.csv')

In [16]:
enhanced_met = enhanced_met[enhanced_met['PatientID'].isin(df_full['PatientID'])]

In [17]:
enhanced_met.loc[:, 'MetDiagnosisDate'] = pd.to_datetime(enhanced_met['MetDiagnosisDate'])

In [18]:
enhanced_met.loc[:, 'CRPCDate'] = pd.to_datetime(enhanced_met['CRPCDate'])

In [19]:
df_full = pd.merge(df_full, enhanced_met[['PatientID', 'MetDiagnosisDate', 'CRPCDate']], on = 'PatientID', how = 'left')

In [20]:
row_ID(df_full)

(18927, 18927)

In [21]:
# Find all that start ADT within -120 to +90 days of metastatic diagnosis 
chaarted_adt = (
    df_full
    .assign(adt_diff = (df_full['StartDate_adt'] - df_full['MetDiagnosisDate']).dt.days)
    .query('adt_diff >= -120 and adt_diff <= 90')
)

In [22]:
# Find all that have missing CPRC date or date is >90 after metastatic diagnosis 
chaarted_adt = (
    chaarted_adt
    .assign(crpc_diff = (chaarted_adt['CRPCDate'] - chaarted_adt['MetDiagnosisDate']).dt.days)
    .query('crpc_diff > 90 or CRPCDate.isna()', engine = 'python')
)

In [23]:
row_ID(chaarted_adt)

(10475, 10475)

In [24]:
line_therapy = pd.read_csv('LineOfTherapy.csv')

In [25]:
zero = (
    line_therapy.query('LineNumber == 0')
    .PatientID
)

In [26]:
# Exclude patients with missing treatment information (ie, LineNumber == 0)
chaarted_adt = chaarted_adt[~chaarted_adt.PatientID.isin(zero)]

In [27]:
row_ID(chaarted_adt)

(10117, 10117)

In [28]:
line_therapy_cont = line_therapy.query('LineSetting != "nmCRPC"')

In [29]:
# List of FDA approved drugs for mPC as of July 2023. Clinical study drug is also included. 
fda_yes = [
    'Abiraterone',
    'Apalutamide',
    'Cabazitaxel',
    'Carboplatin',
    'Cisplatin',
    'Darolutamide',
    'Docetaxel',
    'Enzalutamide',
    'Mitoxantrone',
    'Olaparib',
    'Oxaliplatin',
    'Paclitaxel',
    'Pembrolizumab',
    'Radium-223',
    'Rucaparib',
    'Sipuleucel-T',
    'Clinical Study Drug'
]

In [30]:
line_therapy_cont = line_therapy_cont[line_therapy_cont['LineName'].str.contains('|'.join(fda_yes))]

In [31]:
line_therapy_cont = (
    line_therapy_cont
    .sort_values(by = ['PatientID', 'StartDate'], ascending = [True, True])
)

In [32]:
line_therapy_cont['line_number'] = (
    line_therapy_cont.groupby('PatientID')['LineNumber'].cumcount()+1
)

In [33]:
# First line therapy is in castrate-resistant setting 
fl_crpc = (
    line_therapy_cont[line_therapy_cont.PatientID.isin(chaarted_adt.PatientID)]
    .query('line_number == 1 & LineSetting == "mCRPC"')
    .PatientID
)

In [34]:
# Never received therapy other than ADT
notrt_adt = (
    chaarted_adt[~chaarted_adt.PatientID.isin(line_therapy_cont.PatientID)]
    .PatientID
)

In [35]:
adt_IDs = np.concatenate((fl_crpc, notrt_adt))

In [36]:
len(adt_IDs)

6218

In [37]:
chaarted_adt = chaarted_adt[chaarted_adt.PatientID.isin(adt_IDs)]

In [38]:
chaarted_adt.loc[:,'adt_dotx'] = 0

In [39]:
row_ID(chaarted_adt)

(6218, 6218)

In [40]:
chaarted_adt.sample(3)

,PatientID,Gender,race,ethnicity,age,p_type,NStage,MStage,Histology,GleasonScore,...,other_met,prim_treatment,early_adt,risk_score,StartDate_adt,MetDiagnosisDate,CRPCDate,adt_diff,crpc_diff,adt_dotx
16290,F925D52721696,M,white,unknown,76,COMMUNITY,N0,M1,Adenocarcinoma,9,...,0.0,unknown,0.0,-0.249583,2013-11-11,2013-11-11,2015-09-24,0.0,682.0,0
2539,F3A8092C370A4,M,black,unknown,69,COMMUNITY,Unknown / Not documented,M1,Adenocarcinoma,9,...,0.0,unknown,0.0,-0.440295,2018-01-01,2017-11-14,2018-09-14,48.0,304.0,0
8553,FE79569643E14,M,white,unknown,67,COMMUNITY,Unknown / Not documented,M1,Adenocarcinoma,9,...,0.0,unknown,0.0,-0.546585,2017-02-10,2017-01-09,2020-04-16,32.0,1193.0,0


#### Docetaxel + ADT

In [41]:
# Find those that start ADT within -120 to 90 days of metastaic diagnosis 
chaarted_dotx = (
    df_full
    .assign(adt_diff = (df_full['StartDate_adt'] - df_full['MetDiagnosisDate']).dt.days)
    .query('adt_diff >= -120 and adt_diff <= 90')
)

In [42]:
# Find all that have missing CPRC date or date is >90 days metastatic diagnosis 
chaarted_dotx = (
    chaarted_dotx
    .assign(crpc_diff = (chaarted_dotx['CRPCDate'] - chaarted_dotx['MetDiagnosisDate']).dt.days)
    .query('crpc_diff > 90 or CRPCDate.isna()', engine = 'python')
)

In [43]:
row_ID(chaarted_dotx)

(10475, 10475)

In [44]:
# Find start time of first line of mHSPC therapy. 
line_therapy_fl = (
    line_therapy[line_therapy['PatientID'].isin(chaarted_dotx['PatientID'])]
    .query('LineSetting == "mHSPC"')
    .sort_values(by = ['PatientID', 'StartDate'], ascending = [True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .rename(columns = {'StartDate': 'StartDate_dotx'})
)

In [45]:
row_ID(line_therapy_fl)

(4042, 4042)

In [46]:
line_therapy_fl.loc[:, 'StartDate_dotx'] = pd.to_datetime(line_therapy_fl['StartDate_dotx'])

In [47]:
line_therapy_fl[line_therapy_fl['LineName'].str.contains('Docetaxel')].LineName.value_counts().head(10)

Docetaxel                                         1363
Abiraterone,Docetaxel                               27
Carboplatin,Docetaxel                               12
Clinical Study Drug,Docetaxel                       11
Darolutamide,Docetaxel                               9
Docetaxel,Enzalutamide                               6
Carboplatin,Docetaxel,Estramustine                   5
Docetaxel,Ketoconazole                               3
Apalutamide,Docetaxel                                2
Carboplatin,Docetaxel,Estramustine,Thalidomide       2
Name: LineName, dtype: int64

In [48]:
line_dotx = line_therapy_fl.query('LineName == "Docetaxel"')

In [49]:
row_ID(line_dotx)

(1363, 1363)

In [50]:
chaarted_dotx = pd.merge(chaarted_dotx, line_dotx[['PatientID', 'StartDate_dotx']], on = 'PatientID', how = 'left')

In [51]:
row_ID(chaarted_dotx)

(10475, 10475)

In [52]:
# Find all patients that start enzalutamide within 180 days of metastatic diagnosis
chaarted_dotx = (
    chaarted_dotx
    .assign(fl_diff = (chaarted_dotx['StartDate_dotx'] - chaarted_dotx['StartDate_adt']).dt.days)
    .query('fl_diff >= -90 and fl_diff <= 90')
)

In [53]:
len(chaarted_dotx)

1179

In [54]:
chaarted_dotx.sample(3)

,PatientID,Gender,race,ethnicity,age,p_type,NStage,MStage,Histology,GleasonScore,...,prim_treatment,early_adt,risk_score,StartDate_adt,MetDiagnosisDate,CRPCDate,adt_diff,crpc_diff,StartDate_dotx,fl_diff
3866,FC90BF7A88A93,M,white,unknown,63,COMMUNITY,Unknown / Not documented,M1,Adenocarcinoma,9,...,unknown,0.0,-1.078815,2017-10-24,2017-08-23,2019-02-06,62.0,532.0,2017-11-09,16.0
3675,F7E1BDBF9255A,M,white,unknown,63,COMMUNITY,N1,M1b,Adenocarcinoma,9,...,unknown,0.0,-0.919761,2019-10-31,2019-11-04,2021-04-06,-4.0,519.0,2020-01-17,78.0
445,F8EDE0BEBC892,M,white,unknown,66,COMMUNITY,Unknown / Not documented,M1,Adenocarcinoma,Unknown / Not documented,...,unknown,0.0,-0.559730,2019-10-18,2019-10-18,NaT,0.0,NaN,2019-12-02,45.0


In [55]:
chaarted_dotx.loc[:,'adt_dotx'] = 1

In [56]:
chaarted = pd.concat([chaarted_adt, chaarted_dotx], ignore_index = True)

In [57]:
row_ID(chaarted)

(7397, 7397)

In [58]:
chaarted.adt_dotx.value_counts(dropna = False)

0    6218
1    1179
Name: adt_dotx, dtype: int64

In [59]:
chaarted.sample(3)

,PatientID,Gender,race,ethnicity,age,p_type,NStage,MStage,Histology,GleasonScore,...,early_adt,risk_score,StartDate_adt,MetDiagnosisDate,CRPCDate,adt_diff,crpc_diff,adt_dotx,StartDate_dotx,fl_diff
4822,F2CD84E82BF6C,M,white,unknown,45,COMMUNITY,N0,M1b,Adenocarcinoma,8,...,0.0,-0.850780,2019-09-19,2019-07-02,2020-11-01,79.0,488.0,0,NaT,NaN
3554,FD1E2BB8483E3,M,black,unknown,57,COMMUNITY,N1,M1b,Adenocarcinoma,9,...,0.0,-0.489399,2018-03-14,2018-03-14,2019-07-16,0.0,489.0,0,NaT,NaN
948,F36D85604756D,M,white,unknown,61,ACADEMIC,Unknown / Not documented,M1,Adenocarcinoma,9,...,0.0,-0.650088,2018-05-01,2018-02-23,2019-04-17,67.0,418.0,0,NaT,NaN


#### Time from ADT treatment to death or censor 

In [60]:
mortality_tr = pd.read_csv('mortality_cleaned_tr.csv')

In [61]:
mortality_te = pd.read_csv('mortality_cleaned_te.csv')

In [62]:
mortality_tr = mortality_tr[['PatientID', 'death_date', 'last_activity']]

In [63]:
mortality_te = mortality_te[['PatientID', 'death_date', 'last_activity']]

In [64]:
mortality = pd.concat([mortality_tr, mortality_te], ignore_index = True)
row_ID(mortality)

(18927, 18927)

In [65]:
mortality.loc[:, 'last_activity'] = pd.to_datetime(mortality['last_activity'])

In [66]:
mortality.loc[:, 'death_date'] = pd.to_datetime(mortality['death_date'])

In [67]:
len(mortality)

18927

In [68]:
chaarted = pd.merge(chaarted, mortality, on = 'PatientID', how = 'left')

In [69]:
row_ID(chaarted)

(7397, 7397)

In [70]:
conditions = [
    (chaarted['death_status'] == 1),
    (chaarted['death_status'] == 0)]

choices = [
    (chaarted['death_date'] - chaarted['StartDate_adt']).dt.days,
    (chaarted['last_activity'] - chaarted['StartDate_adt']).dt.days]

chaarted.loc[:, 'timerisk_treatment'] = np.select(conditions, choices)

chaarted = chaarted.query('timerisk_treatment >= 0')

#### Patient count 

In [71]:
low_cutoff_chaarted = chaarted.risk_score.quantile(1/3)

In [72]:
high_cutoff_chaarted = chaarted.risk_score.quantile(2/3)

In [73]:
print('Docetaxel + ADT:',  chaarted.query('adt_dotx == 1').shape[0])
print('High risk:', chaarted.query('adt_dotx == 1').query('risk_score >= @high_cutoff_chaarted').shape[0])
print('Med risk:', chaarted.query('adt_dotx == 1').query('risk_score < @high_cutoff_chaarted and risk_score > @low_cutoff_chaarted').shape[0])
print('Low risk:', chaarted.query('adt_dotx == 1').query('risk_score <= @low_cutoff_chaarted').shape[0])

Docetaxel + ADT: 1179
High risk: 316
Med risk: 424
Low risk: 439


In [74]:
print('ADT:',  chaarted.query('adt_dotx == 0').shape[0])
print('High risk:', chaarted.query('adt_dotx == 0').query('risk_score >= @high_cutoff_chaarted').shape[0])
print('Med risk:', chaarted.query('adt_dotx == 0').query('risk_score < @high_cutoff_chaarted and risk_score > @low_cutoff_chaarted').shape[0])
print('Low risk:', chaarted.query('adt_dotx == 0').query('risk_score <= @low_cutoff_chaarted').shape[0])

ADT: 6218
High risk: 2150
Med risk: 2041
Low risk: 2027


#### Survival with covariate balancing 

In [75]:
chaarted = chaarted.set_index('PatientID')

In [76]:
chaarted['met_cat'] = pd.cut(chaarted['met_year'],
                             bins = [2010, 2015, float('inf')],
                             labels = ['11-15', '16-22'])

In [77]:
conditions = [
    ((chaarted['ecog_diagnosis'] == "1.0") | (chaarted['ecog_diagnosis'] == "0.0")),  
    ((chaarted['ecog_diagnosis'] == "2.0") | (chaarted['ecog_diagnosis'] == "3.0"))
]

choices = ['lt_2', 'gte_2']

chaarted['ecog_2'] = np.select(conditions, choices, default = 'unknown')

In [78]:
chaarted_iptw = chaarted.filter(items = ['death_status',
                                         'timerisk_treatment',
                                         'adt_dotx',
                                         'age',
                                         'race',
                                         'p_type',
                                         'met_cat',
                                         'delta_met_diagnosis',
                                         'commercial',
                                         'medicare',
                                         'medicaid',
                                         'ecog_2',
                                         'prim_treatment',
                                         'PSAMetDiagnosis',
                                         'albumin_diag', 
                                         'weight_pct_change',
                                         'risk_score'])

In [79]:
chaarted_iptw.dtypes

death_status               bool
timerisk_treatment      float64
adt_dotx                  int64
age                       int64
race                     object
p_type                   object
met_cat                category
delta_met_diagnosis       int64
commercial              float64
medicare                float64
medicaid                float64
ecog_2                   object
prim_treatment           object
PSAMetDiagnosis         float64
albumin_diag            float64
weight_pct_change       float64
risk_score              float64
dtype: object

In [80]:
to_be_categorical = list(chaarted_iptw.select_dtypes(include = ['object']).columns)

In [81]:
to_be_categorical

['race', 'p_type', 'ecog_2', 'prim_treatment']

In [82]:
to_be_categorical.append('met_cat')

In [83]:
# Convert variables in list to categorical.
for x in list(to_be_categorical):
    chaarted_iptw[x] = chaarted_iptw[x].astype('category')

In [84]:
# List of numeric variables, excluding binary variables. 
numerical_features = ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score']

# Transformer will first calculate column median and impute, and then apply a standard scaler. 
numerical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('std_scaler', StandardScaler())])

In [85]:
# List of categorical features.
categorical_features = list(chaarted_iptw.select_dtypes(include = ['category']).columns)

# One-hot-encode categorical features.
categorical_transformer = OneHotEncoder(handle_unknown = 'ignore')

In [86]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder = 'passthrough')

In [87]:
chaarted_iptw_low = (
    chaarted_iptw
    .query('risk_score <= @low_cutoff_chaarted'))

chaarted_iptw_med = (
    chaarted_iptw
    .query('risk_score < @high_cutoff_chaarted and risk_score > @low_cutoff_chaarted'))

chaarted_iptw_high = (
    chaarted_iptw
    .query('risk_score >= @high_cutoff_chaarted'))

chaarted_iptw_all = chaarted_iptw

In [88]:
chaarted_low_x = preprocessor.fit_transform(chaarted_iptw_low.filter(items = ['age',
                                                                              'race',
                                                                              'p_type',
                                                                              'delta_met_diagnosis',
                                                                              'met_cat',
                                                                              'commercial',
                                                                              'medicare',
                                                                              'medicaid',
                                                                              'ecog_2',
                                                                              'prim_treatment',
                                                                              'PSAMetDiagnosis', 
                                                                              'albumin_diag', 
                                                                              'weight_pct_change',
                                                                              'risk_score']))

chaarted_med_x = preprocessor.fit_transform(chaarted_iptw_med.filter(items = ['age',
                                                                              'race',
                                                                              'p_type',
                                                                              'delta_met_diagnosis',
                                                                              'met_cat',
                                                                              'commercial',
                                                                              'medicare',
                                                                              'medicaid',
                                                                              'ecog_2',
                                                                              'prim_treatment',
                                                                              'PSAMetDiagnosis', 
                                                                              'albumin_diag', 
                                                                              'weight_pct_change', 
                                                                              'risk_score']))

chaarted_high_x = preprocessor.fit_transform(chaarted_iptw_high.filter(items = ['age',
                                                                                'race',
                                                                                'p_type',
                                                                                'delta_met_diagnosis',
                                                                                'met_cat',
                                                                                'commercial',
                                                                                'medicare',
                                                                                'medicaid',
                                                                                'ecog_2',
                                                                                'prim_treatment',
                                                                                'PSAMetDiagnosis', 
                                                                                'albumin_diag', 
                                                                                'weight_pct_change',
                                                                                'risk_score']))

chaarted_all_x = preprocessor.fit_transform(chaarted_iptw_all.filter(items = ['age',
                                                                              'race',
                                                                              'p_type',
                                                                              'delta_met_diagnosis',
                                                                              'met_cat',
                                                                              'commercial',
                                                                              'medicare',
                                                                              'medicaid',
                                                                              'ecog_2',
                                                                              'prim_treatment',
                                                                              'PSAMetDiagnosis', 
                                                                              'albumin_diag', 
                                                                              'weight_pct_change', 
                                                                              'risk_score']))

In [89]:
lr_chaarted_low = LogisticRegression(max_iter = 1000)
lr_chaarted_low.fit(chaarted_low_x, chaarted_iptw_low['adt_dotx'])

LogisticRegression(max_iter=1000)

In [90]:
lr_chaarted_med = LogisticRegression(max_iter = 1000)
lr_chaarted_med.fit(chaarted_med_x, chaarted_iptw_med['adt_dotx'])

LogisticRegression(max_iter=1000)

In [91]:
lr_chaarted_high = LogisticRegression(max_iter = 1000)
lr_chaarted_high.fit(chaarted_high_x, chaarted_iptw_high['adt_dotx'])

LogisticRegression(max_iter=1000)

In [92]:
lr_chaarted_all = LogisticRegression(max_iter = 1000)
lr_chaarted_all.fit(chaarted_all_x, chaarted_iptw_all['adt_dotx'])

LogisticRegression(max_iter=1000)

In [93]:
pred_low = lr_chaarted_low.predict_proba(chaarted_low_x)
pred_med = lr_chaarted_med.predict_proba(chaarted_med_x)
pred_high = lr_chaarted_high.predict_proba(chaarted_high_x)
pred_all = lr_chaarted_all.predict_proba(chaarted_all_x)

In [94]:
chaarted_iptw_low['ps'] = pred_low[:, 1]
chaarted_iptw_med['ps'] = pred_med[:, 1]
chaarted_iptw_high['ps'] = pred_high[:, 1]
chaarted_iptw_all['ps'] = pred_all[:, 1]

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [95]:
chaarted_iptw_low['weight'] = (
    np.where(chaarted_iptw_low['adt_dotx'] == 1, 1/chaarted_iptw_low['ps'], 1/(1 - chaarted_iptw_low['ps'])))

chaarted_iptw_med['weight'] = (
    np.where(chaarted_iptw_med['adt_dotx'] == 1, 1/chaarted_iptw_med['ps'], 1/(1 - chaarted_iptw_med['ps'])))

chaarted_iptw_high['weight'] = (
    np.where(chaarted_iptw_high['adt_dotx'] == 1, 1/chaarted_iptw_high['ps'], 1/(1 - chaarted_iptw_high['ps'])))

chaarted_iptw_all['weight'] = (
    np.where(chaarted_iptw_all['adt_dotx'] == 1, 1/chaarted_iptw_all['ps'], 1/(1 - chaarted_iptw_all['ps'])))

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [96]:
# Low KM curves
kmf_low_dotx_chaarted_iptw = KaplanMeierFitter()
kmf_low_adt_chaarted_iptw = KaplanMeierFitter()

kmf_low_dotx_chaarted_iptw.fit(
    chaarted_iptw_low.query('adt_dotx == 1').timerisk_treatment/30,
    chaarted_iptw_low.query('adt_dotx == 1').death_status,
    weights = chaarted_iptw_low.query('adt_dotx == 1')['weight'])

kmf_low_adt_chaarted_iptw.fit(
    chaarted_iptw_low.query('adt_dotx == 0').timerisk_treatment/30,
    chaarted_iptw_low.query('adt_dotx == 0').death_status,
    weights = chaarted_iptw_low.query('adt_dotx == 0')['weight'])

# Med KM curves
kmf_med_dotx_chaarted_iptw = KaplanMeierFitter()
kmf_med_adt_chaarted_iptw = KaplanMeierFitter()

kmf_med_dotx_chaarted_iptw.fit(
    chaarted_iptw_med.query('adt_dotx == 1').timerisk_treatment/30,
    chaarted_iptw_med.query('adt_dotx == 1').death_status,
    weights = chaarted_iptw_med.query('adt_dotx == 1')['weight'])

kmf_med_adt_chaarted_iptw.fit(
    chaarted_iptw_med.query('adt_dotx == 0').timerisk_treatment/30,
    chaarted_iptw_med.query('adt_dotx == 0').death_status,
    weights = chaarted_iptw_med.query('adt_dotx == 0')['weight'])

# High KM curves 
kmf_high_dotx_chaarted_iptw = KaplanMeierFitter()
kmf_high_adt_chaarted_iptw = KaplanMeierFitter()

kmf_high_dotx_chaarted_iptw.fit(
    chaarted_iptw_high.query('adt_dotx == 1').timerisk_treatment/30,
    chaarted_iptw_high.query('adt_dotx == 1').death_status,
    weights = chaarted_iptw_high.query('adt_dotx == 1')['weight'])

kmf_high_adt_chaarted_iptw.fit(
    chaarted_iptw_high.query('adt_dotx == 0').timerisk_treatment/30,
    chaarted_iptw_high.query('adt_dotx == 0').death_status,
    weights = chaarted_iptw_high.query('adt_dotx == 0')['weight'])

# All KM curves 
kmf_all_dotx_chaarted_iptw = KaplanMeierFitter()
kmf_all_adt_chaarted_iptw = KaplanMeierFitter()

kmf_all_dotx_chaarted_iptw.fit(
    chaarted_iptw_all.query('adt_dotx == 1').timerisk_treatment/30,
    chaarted_iptw_all.query('adt_dotx == 1').death_status,
    weights = chaarted_iptw_all.query('adt_dotx == 1')['weight'])

kmf_all_adt_chaarted_iptw.fit(
    chaarted_iptw_all.query('adt_dotx == 0').timerisk_treatment/30,
    chaarted_iptw_all.query('adt_dotx == 0').death_status,
    weights = chaarted_iptw_all.query('adt_dotx == 0')['weight'])

/Users/xavierorcutt/Dropbox/ist_project/myenv/lib/python3.7/site-packages/lifelines/fitters/kaplan_meier_fitter.py:344: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  StatisticalWarning,


<lifelines.KaplanMeierFitter:"KM_estimate", fitted with 7415.3 total observations, 3343.73 right-censored observations>

#### Calculating survival metrics 

In [97]:
dotx_chaarted_median_os = mos(kmf_low_dotx_chaarted_iptw,
                              kmf_med_dotx_chaarted_iptw,
                              kmf_high_dotx_chaarted_iptw,
                              kmf_all_dotx_chaarted_iptw)

adt_chaarted_median_os = mos(kmf_low_adt_chaarted_iptw,
                             kmf_med_adt_chaarted_iptw,
                             kmf_high_adt_chaarted_iptw,
                             kmf_all_adt_chaarted_iptw)

In [98]:
chaarted_iptw_all_imputed = chaarted_iptw_all.copy()
chaarted_iptw_all_imputed['albumin_diag'] = chaarted_iptw_all_imputed['albumin_diag'].fillna(chaarted_iptw_all_imputed['albumin_diag'].median())
chaarted_iptw_all_imputed['weight_pct_change'] = chaarted_iptw_all_imputed['weight_pct_change'].fillna(chaarted_iptw_all_imputed['weight_pct_change'].median())
chaarted_iptw_all_imputed['PSAMetDiagnosis'] = chaarted_iptw_all_imputed['PSAMetDiagnosis'].fillna(chaarted_iptw_all_imputed['PSAMetDiagnosis'].median())

In [99]:
chaarted_hr_all = CoxPHFitter()
chaarted_hr_all.fit(chaarted_iptw_all_imputed,
                    duration_col = 'timerisk_treatment',
                    event_col = 'death_status',
                    formula = 'adt_dotx + age + race + p_type + delta_met_diagnosis + met_cat + commercial + medicare + medicaid + ecog_2 + prim_treatment + PSAMetDiagnosis + albumin_diag + weight_pct_change + risk_score',
                    weights_col = 'weight', 
                    robust = True)

<lifelines.CoxPHFitter: fitted with 14798.3 total observations, 7094.15 right-censored observations>

In [100]:
chaarted_all_rmst_mos_95 = rmst_mos_95ci(chaarted_iptw_all,
                                         1000,
                                         'adt_dotx',
                                         'death',
                                         ['age',
                                          'race',
                                          'p_type',
                                          'delta_met_diagnosis',
                                          'met_cat',
                                          'commercial',
                                          'medicare',
                                          'medicaid',
                                          'ecog_2',
                                          'prim_treatment',
                                          'PSAMetDiagnosis', 
                                          'albumin_diag', 
                                          'weight_pct_change',
                                          'risk_score'],
                                         ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score'],
                                         60)

In [101]:
chaarted_low_rmst_mos_95 = rmst_mos_95ci(chaarted_iptw_low,
                                         1000,
                                         'adt_dotx',
                                         'death',
                                         ['age',
                                          'race',
                                          'p_type',
                                          'delta_met_diagnosis',
                                          'met_cat',
                                          'commercial',
                                          'medicare',
                                          'medicaid',
                                          'ecog_2',
                                          'prim_treatment',
                                          'PSAMetDiagnosis', 
                                          'albumin_diag', 
                                          'weight_pct_change',
                                          'risk_score'],
                                         ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score'],
                                         60)

In [102]:
chaarted_med_rmst_mos_95 = rmst_mos_95ci(chaarted_iptw_med,
                                         1000,
                                         'adt_dotx',
                                         'death',
                                         ['age',
                                          'race',
                                          'p_type',
                                          'delta_met_diagnosis',
                                          'met_cat',
                                          'commercial',
                                          'medicare',
                                          'medicaid',
                                          'ecog_2',
                                          'prim_treatment',
                                          'PSAMetDiagnosis', 
                                          'albumin_diag', 
                                          'weight_pct_change',
                                          'risk_score'],
                                         ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score'],
                                         60)

In [103]:
chaarted_high_rmst_mos_95 = rmst_mos_95ci(chaarted_iptw_high,
                                          1000,
                                          'adt_dotx',
                                          'death',
                                          ['age',
                                           'race',
                                           'p_type',
                                           'delta_met_diagnosis',
                                           'met_cat',
                                           'commercial',
                                           'medicare',
                                           'medicaid',
                                           'ecog_2',
                                           'prim_treatment',
                                           'PSAMetDiagnosis', 
                                           'albumin_diag', 
                                           'weight_pct_change',
                                           'risk_score'],
                                          ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score'],
                                          60)

In [104]:
chaarted_data = [
    {'trial_name': 'CHAARTED', 
     'risk_group': 'low', 
     'r_trt_mos': dotx_chaarted_median_os[0],
     'r_trt_mos_95': chaarted_low_rmst_mos_95.mos_A_95,
     'r_cont_mos': adt_chaarted_median_os[0],
     'r_cont_mos_95': chaarted_low_rmst_mos_95.mos_B_95,
     'r_mos_diff': dotx_chaarted_median_os[0] - adt_chaarted_median_os[0], 
     'rct_trt_arm': 57.6,
     'rct_cont_arm': 44.0,
     'rct_mos_diff': 57.6-44.0,
     'trt_rmst': restricted_mean_survival_time(kmf_low_dotx_chaarted_iptw, 60),
     'trt_rmst_95': chaarted_low_rmst_mos_95.rmst_A_95,
     'cont_rmst': restricted_mean_survival_time(kmf_low_adt_chaarted_iptw, 60),
     'cont_rmst_95': chaarted_low_rmst_mos_95.rmst_B_95,
     'diff_rmst': restricted_mean_survival_time(kmf_low_dotx_chaarted_iptw, 60) - restricted_mean_survival_time(kmf_low_adt_chaarted_iptw, 60),
     'diff_rmst_95': chaarted_low_rmst_mos_95.difference_rmst_95,
     'rcount': chaarted.query('risk_score <= @low_cutoff_chaarted').shape[0],
     'rcount_chemo': chaarted.query('risk_score <= @low_cutoff_chaarted').query('adt_dotx == 1').shape[0]},
    
    {'trial_name': 'CHAARTED', 
     'risk_group': 'medium', 
     'r_trt_mos': dotx_chaarted_median_os[1],
     'r_trt_mos_95': chaarted_med_rmst_mos_95.mos_A_95,
     'r_cont_mos': adt_chaarted_median_os[1],
     'r_cont_mos_95': chaarted_med_rmst_mos_95.mos_B_95,
     'r_mos_diff': dotx_chaarted_median_os[1] - adt_chaarted_median_os[1], 
     'rct_trt_arm': 57.6,
     'rct_cont_arm': 44.0,
     'rct_mos_diff': 57.6-44.0,
     'trt_rmst': restricted_mean_survival_time(kmf_med_dotx_chaarted_iptw, 60),
     'trt_rmst_95': chaarted_med_rmst_mos_95.rmst_A_95,
     'cont_rmst': restricted_mean_survival_time(kmf_med_adt_chaarted_iptw, 60),
     'cont_rmst_95': chaarted_med_rmst_mos_95.rmst_B_95,
     'diff_rmst': restricted_mean_survival_time(kmf_med_dotx_chaarted_iptw, 60) - restricted_mean_survival_time(kmf_med_adt_chaarted_iptw, 60),
     'diff_rmst_95': chaarted_med_rmst_mos_95.difference_rmst_95,
     'rcount': chaarted.query('risk_score < @high_cutoff_chaarted and risk_score > @low_cutoff_chaarted').shape[0],
     'rcount_chemo': chaarted.query('risk_score < @high_cutoff_chaarted and risk_score > @low_cutoff_chaarted').query('adt_dotx == 1').shape[0]},
    
    {'trial_name': 'CHAARTED', 
     'risk_group': 'high', 
     'r_trt_mos': dotx_chaarted_median_os[2],
     'r_trt_mos_95': chaarted_high_rmst_mos_95.mos_A_95,
     'r_cont_mos': adt_chaarted_median_os[2],
     'r_cont_mos_95': chaarted_high_rmst_mos_95.mos_B_95,
     'r_mos_diff': dotx_chaarted_median_os[2] - adt_chaarted_median_os[2], 
     'rct_trt_arm': 57.6,
     'rct_cont_arm': 44.0,
     'rct_mos_diff': 57.6-44.0,
     'trt_rmst': restricted_mean_survival_time(kmf_high_dotx_chaarted_iptw, 60),
     'trt_rmst_95': chaarted_high_rmst_mos_95.rmst_A_95,
     'cont_rmst': restricted_mean_survival_time(kmf_high_adt_chaarted_iptw, 60),
     'cont_rmst_95': chaarted_high_rmst_mos_95.rmst_B_95,
     'diff_rmst': restricted_mean_survival_time(kmf_high_dotx_chaarted_iptw, 60) - restricted_mean_survival_time(kmf_high_adt_chaarted_iptw, 60),
     'diff_rmst_95': chaarted_high_rmst_mos_95.difference_rmst_95,
     'rcount': chaarted.query('risk_score >= @high_cutoff_chaarted').shape[0],
     'rcount_chemo': chaarted.query('risk_score >= @high_cutoff_chaarted').query('adt_dotx == 1').shape[0]},
    
    {'trial_name': 'CHAARTED', 
     'risk_group': 'all', 
     'r_hr': chaarted_hr_all.hazard_ratios_['adt_dotx'],
     'r_hr_95': [chaarted_hr_all.summary.loc['adt_dotx']['exp(coef) lower 95%'], chaarted_hr_all.summary.loc['adt_dotx']['exp(coef) upper 95%']],
     'r_trt_mos': dotx_chaarted_median_os[3],
     'r_trt_mos_95': chaarted_all_rmst_mos_95.mos_A_95,
     'r_cont_mos': adt_chaarted_median_os[3],
     'r_cont_mos_95': chaarted_all_rmst_mos_95.mos_B_95,
     'r_mos_diff': dotx_chaarted_median_os[3] - adt_chaarted_median_os[3], 
     'rct_trt_arm': 57.6,
     'rct_cont_arm': 44.0,
     'rct_mos_diff': 57.6-44.0,
     'rcount': chaarted.shape[0], 
     'rcount_chemo': chaarted.query('adt_dotx == 1').shape[0]}
]

### LATITUDE : abiraterone vs. ADT in metastatic, castration-sensitive prostate cancer  

**INCLUSION**
* Untreated metastatic prostate cancer, except up to 3 months of ADT 
* Castration-sensitive
* Received ADT or abiraterone plus ADT

#### ADT

In [105]:
df_full = pd.read_csv('df_risk_crude.csv', index_col = 'PatientID', dtype = {'death_status': bool})
df_full.index.nunique()

18927

In [106]:
df_full.reset_index(inplace = True)

In [107]:
adt = pd.read_csv('Enhanced_MetPC_ADT.csv')

In [108]:
adt = (
    adt[adt['PatientID'].isin(df_full['PatientID'])]
    .query('TreatmentSetting == "Advanced"')
)

In [109]:
row_ID(adt)

(17863, 17863)

In [110]:
adt.loc[:, 'StartDate'] = pd.to_datetime(adt['StartDate'])

In [111]:
adt = adt.rename(columns = {'StartDate': 'StartDate_adt'})

In [112]:
df_full = pd.merge(df_full, adt[['PatientID', 'StartDate_adt']], on = 'PatientID', how = 'left')

In [113]:
row_ID(df_full)

(18927, 18927)

In [114]:
enhanced_met = pd.read_csv('Enhanced_MetProstate.csv')

In [115]:
enhanced_met = enhanced_met[enhanced_met['PatientID'].isin(df_full['PatientID'])]

In [116]:
enhanced_met.loc[:, 'MetDiagnosisDate'] = pd.to_datetime(enhanced_met['MetDiagnosisDate'])

In [117]:
enhanced_met.loc[:, 'CRPCDate'] = pd.to_datetime(enhanced_met['CRPCDate'])

In [118]:
df_full = pd.merge(df_full, enhanced_met[['PatientID', 'MetDiagnosisDate', 'CRPCDate']], on = 'PatientID', how = 'left')

In [119]:
row_ID(df_full)

(18927, 18927)

In [120]:
# Find all that start ADT within -90 to +90 days of metastatic diagnosis 
latitude_adt = (
    df_full
    .assign(adt_diff = (df_full['StartDate_adt'] - df_full['MetDiagnosisDate']).dt.days)
    .query('adt_diff >= -90 and adt_diff <= 90')
)

In [121]:
# Find all that have missing CPRC date or date is >90 after metastatic diagnosis 
latitude_adt = (
    latitude_adt
    .assign(crpc_diff = (latitude_adt['CRPCDate'] - latitude_adt['MetDiagnosisDate']).dt.days)
    .query('crpc_diff > 90 or CRPCDate.isna()', engine = 'python')
)

In [122]:
row_ID(latitude_adt)

(10397, 10397)

In [123]:
line_therapy = pd.read_csv('LineOfTherapy.csv')

In [124]:
zero = (
    line_therapy.query('LineNumber == 0')
    .PatientID
)

In [125]:
# Exclude patients with missing treatment information (ie, LineNumber == 0)
latitude_adt = latitude_adt[~latitude_adt.PatientID.isin(zero)]

In [126]:
row_ID(latitude_adt)

(10047, 10047)

In [127]:
line_therapy_cont = line_therapy.query('LineSetting != "nmCRPC"')

In [128]:
# List of FDA approved drugs for mPC as of July 2023. Clinical study drug is also included. 
fda_yes = [
    'Abiraterone',
    'Apalutamide',
    'Cabazitaxel',
    'Carboplatin',
    'Cisplatin',
    'Darolutamide',
    'Docetaxel',
    'Enzalutamide',
    'Mitoxantrone',
    'Olaparib',
    'Oxaliplatin',
    'Paclitaxel',
    'Pembrolizumab',
    'Radium-223',
    'Rucaparib',
    'Sipuleucel-T',
    'Clinical Study Drug'
]

In [129]:
line_therapy_cont = line_therapy_cont[line_therapy_cont['LineName'].str.contains('|'.join(fda_yes))]

In [130]:
line_therapy_cont = (
    line_therapy_cont
    .sort_values(by = ['PatientID', 'StartDate'], ascending = [True, True])
)

In [131]:
line_therapy_cont['line_number'] = (
    line_therapy_cont.groupby('PatientID')['LineNumber'].cumcount()+1
)

In [132]:
# First line therapy is in castrate-resistant setting 
fl_crpc = (
    line_therapy_cont[line_therapy_cont.PatientID.isin(latitude_adt.PatientID)]
    .query('line_number == 1 & LineSetting == "mCRPC"')
    .PatientID
)

In [133]:
# Never received therapy other than ADT
notrt_adt = (
    latitude_adt[~latitude_adt.PatientID.isin(line_therapy_cont.PatientID)]
    .PatientID
)

In [134]:
adt_IDs = np.concatenate((fl_crpc, notrt_adt))

In [135]:
len(adt_IDs)

6167

In [136]:
latitude_adt = latitude_adt[latitude_adt.PatientID.isin(adt_IDs)]

In [137]:
latitude_adt.loc[:,'adt_abi'] = 0

In [138]:
row_ID(latitude_adt)

(6167, 6167)

In [139]:
latitude_adt.sample(3)

,PatientID,Gender,race,ethnicity,age,p_type,NStage,MStage,Histology,GleasonScore,...,other_met,prim_treatment,early_adt,risk_score,StartDate_adt,MetDiagnosisDate,CRPCDate,adt_diff,crpc_diff,adt_abi
14549,F9AEA3E040920,M,white,unknown,84,COMMUNITY,Unknown / Not documented,Unknown / Not documented,"Prostate cancer, NOS",Unknown / Not documented,...,0.0,prostatectomy,0.0,-0.346219,2021-01-01,2021-03-02,NaT,-60.0,NaN,0
5280,F51C00C23D372,M,white,unknown,76,COMMUNITY,N1,M1c,Adenocarcinoma,Unknown / Not documented,...,0.0,unknown,0.0,-0.023407,2013-04-19,2013-04-11,2014-12-09,8.0,607.0,0
4859,F28662F5253C2,M,black,unknown,76,COMMUNITY,N1,M1,Adenocarcinoma,3 + 4 = 7,...,0.0,unknown,0.0,-0.169897,2017-01-19,2016-12-01,NaT,49.0,NaN,0


#### Treatment arm: Abiraterone + ADT

In [140]:
# Find those that start ADT within -90 to 90 days of metastaic diagnosis 
latitude_abi = (
    df_full
    .assign(adt_diff = (df_full['StartDate_adt'] - df_full['MetDiagnosisDate']).dt.days)
    .query('adt_diff >= -90 and adt_diff <= 90')
)

In [141]:
# Find all that have missing CPRC date or date is >30 days metastatic diagnosis 
latitude_abi = (
    latitude_abi
    .assign(crpc_diff = (latitude_abi['CRPCDate'] - latitude_abi['MetDiagnosisDate']).dt.days)
    .query('crpc_diff > 90 or CRPCDate.isna()', engine = 'python')
)

In [142]:
row_ID(latitude_abi)

(10397, 10397)

In [143]:
# Find start time of first line of mHSPC therapy. 
line_therapy_fl = (
    line_therapy[line_therapy['PatientID'].isin(latitude_abi['PatientID'])]
    .query('LineSetting == "mHSPC"')
    .sort_values(by = ['PatientID', 'StartDate'], ascending = [True, True])
    .drop_duplicates(subset = ['PatientID'], keep = 'first')
    .rename(columns = {'StartDate': 'StartDate_abi'})
)

In [144]:
row_ID(line_therapy_fl)

(4022, 4022)

In [145]:
line_therapy_fl.loc[:, 'StartDate_abi'] = pd.to_datetime(line_therapy_fl['StartDate_abi'])

In [146]:
line_therapy_fl[line_therapy_fl['LineName'].str.contains('Abiraterone')].LineName.value_counts().head(10)

Abiraterone                             1165
Abiraterone,Docetaxel                     27
Abiraterone,Enzalutamide                   8
Abiraterone,Clinical Study Drug            4
Abiraterone,Apalutamide                    4
Abiraterone,Apalutamide,Enzalutamide       1
Abiraterone,Capecitabine                   1
Abiraterone,Medroxyprogesterone            1
Abiraterone,Fluorouracil                   1
Abiraterone,Tamoxifen                      1
Name: LineName, dtype: int64

In [147]:
line_abi = line_therapy_fl.query('LineName == "Abiraterone"')

In [148]:
row_ID(line_abi)

(1165, 1165)

In [149]:
latitude_abi = pd.merge(latitude_abi, line_abi[['PatientID', 'StartDate_abi']], on = 'PatientID', how = 'left')

In [150]:
row_ID(latitude_abi)

(10397, 10397)

In [151]:
# Find all patients that start abiraterone within 90 days of ADT start
latitude_abi = (
    latitude_abi
    .assign(abi_diff = (latitude_abi['StartDate_abi'] - latitude_abi['StartDate_adt']).dt.days)
    .query('abi_diff >= -90 and abi_diff <= 90')
)

In [152]:
row_ID(latitude_abi)

(876, 876)

In [153]:
latitude_abi.sample(3)

,PatientID,Gender,race,ethnicity,age,p_type,NStage,MStage,Histology,GleasonScore,...,prim_treatment,early_adt,risk_score,StartDate_adt,MetDiagnosisDate,CRPCDate,adt_diff,crpc_diff,StartDate_abi,abi_diff
6969,F7FBC7FAE0FFD,M,white,unknown,76,COMMUNITY,N1,M1,Adenocarcinoma,10,...,unknown,0.0,0.050464,2019-10-30,2019-09-10,NaT,50.0,NaN,2019-12-23,54.0
7025,F954A609D2920,M,other,unknown,74,BOTH,N1,M1,Adenocarcinoma,Unknown / Not documented,...,unknown,0.0,-0.706843,2020-07-03,2020-06-12,NaT,21.0,NaN,2020-09-16,75.0
6830,F4DD596AD65A9,M,white,unknown,67,COMMUNITY,Unknown / Not documented,M1,Adenocarcinoma,Unknown / Not documented,...,unknown,0.0,-0.135481,2022-02-25,2022-01-03,NaT,53.0,NaN,2022-03-18,21.0


In [154]:
latitude_abi.loc[:,'adt_abi'] = 1

In [155]:
latitude = pd.concat([latitude_adt, latitude_abi], ignore_index = True)

In [156]:
row_ID(latitude)

(7043, 7043)

In [157]:
latitude.adt_abi.value_counts(dropna = False)

0    6167
1     876
Name: adt_abi, dtype: int64

#### Time from ADT treatment to death or censor 

In [158]:
mortality_tr = pd.read_csv('mortality_cleaned_tr.csv')

In [159]:
mortality_te = pd.read_csv('mortality_cleaned_te.csv')

In [160]:
mortality_tr = mortality_tr[['PatientID', 'death_date', 'last_activity']]

In [161]:
mortality_te = mortality_te[['PatientID', 'death_date', 'last_activity']]

In [162]:
mortality = pd.concat([mortality_tr, mortality_te], ignore_index = True)
row_ID(mortality)

(18927, 18927)

In [163]:
mortality.loc[:, 'last_activity'] = pd.to_datetime(mortality['last_activity'])

In [164]:
mortality.loc[:, 'death_date'] = pd.to_datetime(mortality['death_date'])

In [165]:
len(mortality)

18927

In [166]:
latitude = pd.merge(latitude, mortality, on = 'PatientID', how = 'left')

In [167]:
row_ID(latitude)

(7043, 7043)

In [168]:
conditions = [
    (latitude['death_status'] == 1),
    (latitude['death_status'] == 0)]

choices = [
    (latitude['death_date'] - latitude['StartDate_adt']).dt.days,
    (latitude['last_activity'] - latitude['StartDate_adt']).dt.days]

latitude.loc[:, 'timerisk_treatment'] = np.select(conditions, choices)

latitude = latitude.query('timerisk_treatment >= 0')

#### Patient count 

In [169]:
low_cutoff_latitude = latitude.risk_score.quantile(1/3)

In [170]:
high_cutoff_latitude = latitude.risk_score.quantile(2/3)

In [171]:
print('Abiraterone + ADT:',  latitude.query('adt_abi == 1').shape[0])
print('High risk:', latitude.query('adt_abi == 1').query('risk_score >= @high_cutoff_latitude').shape[0])
print('Med risk:', latitude.query('adt_abi == 1').query('risk_score < @high_cutoff_latitude and risk_score > @low_cutoff_latitude').shape[0])
print('Low risk:', latitude.query('adt_abi == 1').query('risk_score <= @low_cutoff_latitude').shape[0])

Abiraterone + ADT: 876
High risk: 275
Med risk: 258
Low risk: 343


In [172]:
print('ADT:',  latitude.query('adt_abi == 0').shape[0])
print('High risk:', latitude.query('adt_abi == 0').query('risk_score >= @high_cutoff_latitude').shape[0])
print('Med risk:', latitude.query('adt_abi == 0').query('risk_score < @high_cutoff_latitude and risk_score > @low_cutoff_latitude').shape[0])
print('Low risk:', latitude.query('adt_abi == 0').query('risk_score <= @low_cutoff_latitude').shape[0])

ADT: 6167
High risk: 2073
Med risk: 2089
Low risk: 2005


#### Survival curves with covariate balancing 

In [173]:
latitude = latitude.set_index('PatientID')

In [174]:
latitude['met_cat'] = pd.cut(latitude['met_year'],
                             bins = [2010, 2018, float('inf')],
                             labels = ['11-18', '19-22'])

In [175]:
conditions = [
    ((latitude['ecog_diagnosis'] == "1.0") | (latitude['ecog_diagnosis'] == "0.0")),  
    ((latitude['ecog_diagnosis'] == "2.0") | (latitude['ecog_diagnosis'] == "3.0"))
]

choices = ['lt_2', 'gte_2']

latitude['ecog_2'] = np.select(conditions, choices, default = 'unknown')

In [176]:
latitude_iptw = latitude.filter(items = ['death_status',
                                         'timerisk_treatment',
                                         'adt_abi',
                                         'age',
                                         'race',
                                         'p_type',
                                         'met_cat',
                                         'delta_met_diagnosis',
                                         'commercial',
                                         'medicare',
                                         'medicaid',
                                         'ecog_2',
                                         'prim_treatment',
                                         'PSAMetDiagnosis',
                                         'albumin_diag', 
                                         'weight_pct_change',
                                         'risk_score'])

In [177]:
latitude_iptw.dtypes

death_status               bool
timerisk_treatment      float64
adt_abi                   int64
age                       int64
race                     object
p_type                   object
met_cat                category
delta_met_diagnosis       int64
commercial              float64
medicare                float64
medicaid                float64
ecog_2                   object
prim_treatment           object
PSAMetDiagnosis         float64
albumin_diag            float64
weight_pct_change       float64
risk_score              float64
dtype: object

In [178]:
to_be_categorical = list(latitude_iptw.select_dtypes(include = ['object']).columns)

In [179]:
to_be_categorical

['race', 'p_type', 'ecog_2', 'prim_treatment']

In [180]:
to_be_categorical.append('met_cat')

In [181]:
# Convert variables in list to categorical.
for x in list(to_be_categorical):
    latitude_iptw[x] = latitude_iptw[x].astype('category')

In [182]:
# List of numeric variables, excluding binary variables. 
numerical_features = ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score']

# Transformer will first calculate column median and impute, and then apply a standard scaler. 
numerical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('std_scaler', StandardScaler())])

In [183]:
# List of categorical features.
categorical_features = list(latitude_iptw.select_dtypes(include = ['category']).columns)

# One-hot-encode categorical features.
categorical_transformer = OneHotEncoder(handle_unknown = 'ignore')

In [184]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder = 'passthrough')

In [185]:
latitude_iptw_low = (
    latitude_iptw
    .query('risk_score <= @low_cutoff_latitude'))

latitude_iptw_med = (
    latitude_iptw
    .query('risk_score < @high_cutoff_latitude and risk_score > @low_cutoff_latitude'))

latitude_iptw_high = (
    latitude_iptw
    .query('risk_score >= @high_cutoff_latitude'))

latitude_iptw_all = latitude_iptw

In [186]:
latitude_low_x = preprocessor.fit_transform(latitude_iptw_low.filter(items = ['age',
                                                                              'race',
                                                                              'p_type',
                                                                              'met_cat',
                                                                              'delta_met_diagnosis',
                                                                              'commercial',
                                                                              'medicare',
                                                                              'medicaid', 
                                                                              'ecog_2',
                                                                              'prim_treatment',
                                                                              'PSAMetDiagnosis', 
                                                                              'albumin_diag',
                                                                              'weight_pct_change',
                                                                              'risk_score']))

latitude_med_x = preprocessor.fit_transform(latitude_iptw_med.filter(items = ['age',
                                                                              'race',
                                                                              'p_type',
                                                                              'met_cat',
                                                                              'delta_met_diagnosis',
                                                                              'commercial',
                                                                              'medicare',
                                                                              'medicaid', 
                                                                              'ecog_2',
                                                                              'prim_treatment',
                                                                              'PSAMetDiagnosis', 
                                                                              'albumin_diag',
                                                                              'weight_pct_change',
                                                                              'risk_score']))

latitude_high_x = preprocessor.fit_transform(latitude_iptw_high.filter(items = ['age',
                                                                                'race',
                                                                                'p_type',
                                                                                'met_cat',
                                                                                'delta_met_diagnosis',
                                                                                'commercial',
                                                                                'medicare',
                                                                                'medicaid', 
                                                                                'ecog_2',
                                                                                'prim_treatment',
                                                                                'PSAMetDiagnosis', 
                                                                                'albumin_diag',
                                                                                'weight_pct_change', 
                                                                                'risk_score']))

latitude_all_x = preprocessor.fit_transform(latitude_iptw_all.filter(items = ['age',
                                                                              'race',
                                                                              'p_type',
                                                                              'met_cat',
                                                                              'delta_met_diagnosis',
                                                                              'commercial',
                                                                              'medicare',
                                                                              'medicaid', 
                                                                              'ecog_2',
                                                                              'prim_treatment',
                                                                              'PSAMetDiagnosis', 
                                                                              'albumin_diag',
                                                                              'weight_pct_change',
                                                                              'risk_score']))

In [187]:
lr_latitude_low = LogisticRegression(max_iter = 1000)
lr_latitude_low.fit(latitude_low_x, latitude_iptw_low['adt_abi'])

LogisticRegression(max_iter=1000)

In [188]:
lr_latitude_med = LogisticRegression(max_iter = 1000)
lr_latitude_med.fit(latitude_med_x, latitude_iptw_med['adt_abi'])

LogisticRegression(max_iter=1000)

In [189]:
lr_latitude_high = LogisticRegression(max_iter = 1000)
lr_latitude_high.fit(latitude_high_x, latitude_iptw_high['adt_abi'])

LogisticRegression(max_iter=1000)

In [190]:
lr_latitude_all = LogisticRegression(max_iter = 1000)
lr_latitude_all.fit(latitude_all_x, latitude_iptw_all['adt_abi'])

LogisticRegression(max_iter=1000)

In [191]:
pred_low = lr_latitude_low.predict_proba(latitude_low_x)
pred_med = lr_latitude_med.predict_proba(latitude_med_x)
pred_high = lr_latitude_high.predict_proba(latitude_high_x)
pred_all = lr_latitude_all.predict_proba(latitude_all_x)

In [192]:
latitude_iptw_low['ps'] = pred_low[:, 1]
latitude_iptw_med['ps'] = pred_med[:, 1]
latitude_iptw_high['ps'] = pred_high[:, 1]
latitude_iptw_all['ps'] = pred_all[:, 1]

In [193]:
latitude_iptw_low['weight'] = (
    np.where(latitude_iptw_low['adt_abi'] == 1, 1/latitude_iptw_low['ps'], 1/(1 - latitude_iptw_low['ps'])))

latitude_iptw_med['weight'] = (
    np.where(latitude_iptw_med['adt_abi'] == 1, 1/latitude_iptw_med['ps'], 1/(1 - latitude_iptw_med['ps'])))

latitude_iptw_high['weight'] = (
    np.where(latitude_iptw_high['adt_abi'] == 1, 1/latitude_iptw_high['ps'], 1/(1 - latitude_iptw_high['ps'])))

latitude_iptw_all['weight'] = (
    np.where(latitude_iptw_all['adt_abi'] == 1, 1/latitude_iptw_all['ps'], 1/(1 - latitude_iptw_all['ps'])))

In [194]:
# Low KM curves
kmf_low_abi_latitude_iptw = KaplanMeierFitter()
kmf_low_adt_latitude_iptw = KaplanMeierFitter()

kmf_low_abi_latitude_iptw.fit(
    latitude_iptw_low.query('adt_abi == 1').timerisk_treatment/30,
    latitude_iptw_low.query('adt_abi == 1').death_status,
    weights = latitude_iptw_low.query('adt_abi == 1')['weight'])

kmf_low_adt_latitude_iptw.fit(
    latitude_iptw_low.query('adt_abi == 0').timerisk_treatment/30,
    latitude_iptw_low.query('adt_abi == 0').death_status,
    weights = latitude_iptw_low.query('adt_abi == 0')['weight'])

# Med KM curves
kmf_med_abi_latitude_iptw = KaplanMeierFitter()
kmf_med_adt_latitude_iptw = KaplanMeierFitter()

kmf_med_abi_latitude_iptw.fit(
    latitude_iptw_med.query('adt_abi == 1').timerisk_treatment/30,
    latitude_iptw_med.query('adt_abi == 1').death_status,
    weights = latitude_iptw_med.query('adt_abi == 1')['weight'])

kmf_med_adt_latitude_iptw.fit(
    latitude_iptw_med.query('adt_abi == 0').timerisk_treatment/30,
    latitude_iptw_med.query('adt_abi == 0').death_status,
    weights = latitude_iptw_med.query('adt_abi == 0')['weight'])

# High KM curves 
kmf_high_abi_latitude_iptw = KaplanMeierFitter()
kmf_high_adt_latitude_iptw = KaplanMeierFitter()

kmf_high_abi_latitude_iptw.fit(
    latitude_iptw_high.query('adt_abi == 1').timerisk_treatment/30,
    latitude_iptw_high.query('adt_abi == 1').death_status,
    weights = latitude_iptw_high.query('adt_abi == 1')['weight'])

kmf_high_adt_latitude_iptw.fit(
    latitude_iptw_high.query('adt_abi == 0').timerisk_treatment/30,
    latitude_iptw_high.query('adt_abi == 0').death_status,
    weights = latitude_iptw_high.query('adt_abi == 0')['weight'])

# All KM curves 
kmf_all_abi_latitude_iptw = KaplanMeierFitter()
kmf_all_adt_latitude_iptw = KaplanMeierFitter()

kmf_all_abi_latitude_iptw.fit(
    latitude_iptw_all.query('adt_abi == 1').timerisk_treatment/30,
    latitude_iptw_all.query('adt_abi == 1').death_status,
    weights = latitude_iptw_all.query('adt_abi == 1')['weight'])

kmf_all_adt_latitude_iptw.fit(
    latitude_iptw_all.query('adt_abi == 0').timerisk_treatment/30,
    latitude_iptw_all.query('adt_abi == 0').death_status,
    weights = latitude_iptw_all.query('adt_abi == 0')['weight'])

<lifelines.KaplanMeierFitter:"KM_estimate", fitted with 7046.86 total observations, 3266.84 right-censored observations>

#### Calculating survival metrics 

In [195]:
abi_latitude_median_os = mos(kmf_low_abi_latitude_iptw,
                             kmf_med_abi_latitude_iptw,
                             kmf_high_abi_latitude_iptw,
                             kmf_all_abi_latitude_iptw)

adt_latitude_median_os = mos(kmf_low_adt_latitude_iptw,
                             kmf_med_adt_latitude_iptw,
                             kmf_high_adt_latitude_iptw,
                             kmf_all_adt_latitude_iptw)

In [196]:
latitude_iptw_all_imputed = latitude_iptw_all.copy()
latitude_iptw_all_imputed['albumin_diag'] = latitude_iptw_all_imputed['albumin_diag'].fillna(latitude_iptw_all_imputed['albumin_diag'].median())
latitude_iptw_all_imputed['weight_pct_change'] = latitude_iptw_all_imputed['weight_pct_change'].fillna(latitude_iptw_all_imputed['weight_pct_change'].median())
latitude_iptw_all_imputed['PSAMetDiagnosis'] = latitude_iptw_all_imputed['PSAMetDiagnosis'].fillna(latitude_iptw_all_imputed['PSAMetDiagnosis'].median())

In [197]:
latitude_hr_all = CoxPHFitter()
latitude_hr_all.fit(latitude_iptw_all_imputed,
                    duration_col = 'timerisk_treatment',
                    event_col = 'death_status',
                    formula = 'adt_abi + age + race + p_type + delta_met_diagnosis + met_cat + commercial + medicare + medicaid + ecog_2 + prim_treatment + PSAMetDiagnosis + albumin_diag + weight_pct_change + risk_score',
                    weights_col = 'weight', 
                    robust = True)

<lifelines.CoxPHFitter: fitted with 13909.9 total observations, 7462.85 right-censored observations>

In [198]:
latitude_all_rmst_mos_95 = rmst_mos_95ci(latitude_iptw_all,
                                         1000,
                                         'adt_abi',
                                         'death',
                                         ['age',
                                          'race',
                                          'p_type',
                                          'delta_met_diagnosis',
                                          'met_cat',
                                          'commercial',
                                          'medicare',
                                          'medicaid',
                                          'ecog_2',
                                          'prim_treatment',
                                          'PSAMetDiagnosis', 
                                          'albumin_diag', 
                                          'weight_pct_change',
                                          'risk_score'],
                                         ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score'],
                                         60)

In [199]:
latitude_low_rmst_mos_95 = rmst_mos_95ci(latitude_iptw_low,
                                         1000,
                                         'adt_abi',
                                         'death',
                                         ['age',
                                          'race',
                                          'p_type',
                                          'delta_met_diagnosis',
                                          'met_cat',
                                          'commercial',
                                          'medicare',
                                          'medicaid',
                                          'ecog_2',
                                          'prim_treatment',
                                          'PSAMetDiagnosis', 
                                          'albumin_diag', 
                                          'weight_pct_change',
                                          'risk_score'],
                                         ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score'],
                                         60)

In [200]:
latitude_med_rmst_mos_95 = rmst_mos_95ci(latitude_iptw_med,
                                         1000,
                                         'adt_abi',
                                         'death',
                                         ['age',
                                          'race',
                                          'p_type',
                                          'delta_met_diagnosis',
                                          'met_cat',
                                          'commercial',
                                          'medicare',
                                          'medicaid',
                                          'ecog_2',
                                          'prim_treatment',
                                          'PSAMetDiagnosis', 
                                          'albumin_diag', 
                                          'weight_pct_change',
                                          'risk_score'],
                                         ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score'],
                                         60)

In [201]:
latitude_high_rmst_mos_95 = rmst_mos_95ci(latitude_iptw_high,
                                          1000,
                                          'adt_abi',
                                          'death',
                                          ['age',
                                           'race',
                                           'p_type',
                                           'delta_met_diagnosis',
                                           'met_cat',
                                           'commercial',
                                           'medicare',
                                           'medicaid',
                                           'ecog_2',
                                           'prim_treatment',
                                           'PSAMetDiagnosis', 
                                           'albumin_diag', 
                                           'weight_pct_change',
                                           'risk_score'],
                                          ['age', 'delta_met_diagnosis', 'PSAMetDiagnosis', 'albumin_diag', 'weight_pct_change', 'risk_score'],
                                          60)

In [202]:
latitude_data = [
    {'trial_name': 'LATITUDE', 
     'risk_group': 'low', 
     'r_trt_mos': abi_latitude_median_os[0],
     'r_trt_mos_95': latitude_low_rmst_mos_95.mos_A_95,
     'r_cont_mos': adt_latitude_median_os[0],
     'r_cont_mos_95': latitude_low_rmst_mos_95.mos_B_95,
     'r_mos_diff': abi_latitude_median_os[0] - adt_latitude_median_os[0], 
     'rct_trt_arm': 53.3,
     'rct_cont_arm': 36.5,
     'rct_mos_diff': 53.3-36.5,
     'trt_rmst': restricted_mean_survival_time(kmf_low_abi_latitude_iptw, 60),
     'trt_rmst_95': latitude_low_rmst_mos_95.rmst_A_95,
     'cont_rmst': restricted_mean_survival_time(kmf_low_adt_latitude_iptw, 60),
     'cont_rmst_95': latitude_low_rmst_mos_95.rmst_B_95,
     'diff_rmst': restricted_mean_survival_time(kmf_low_abi_latitude_iptw, 60) - restricted_mean_survival_time(kmf_low_adt_latitude_iptw, 60),
     'diff_rmst_95': latitude_low_rmst_mos_95.difference_rmst_95,
     'rcount': latitude.query('risk_score <= @low_cutoff_latitude').shape[0]},
    
    {'trial_name': 'LATITUDE', 
     'risk_group': 'medium', 
     'r_trt_mos': abi_latitude_median_os[1],
     'r_trt_mos_95': latitude_med_rmst_mos_95.mos_A_95,
     'r_cont_mos': adt_latitude_median_os[1],
     'r_cont_mos_95': latitude_med_rmst_mos_95.mos_B_95,
     'r_mos_diff': abi_latitude_median_os[1] - adt_latitude_median_os[1], 
     'rct_trt_arm': 53.3,
     'rct_cont_arm': 36.5,
     'rct_mos_diff': 53.3-36.5,
     'trt_rmst': restricted_mean_survival_time(kmf_med_abi_latitude_iptw, 60),
     'trt_rmst_95': latitude_med_rmst_mos_95.rmst_A_95,
     'cont_rmst': restricted_mean_survival_time(kmf_med_adt_latitude_iptw, 60),
     'cont_rmst_95': latitude_med_rmst_mos_95.rmst_B_95,
     'diff_rmst': restricted_mean_survival_time(kmf_med_abi_latitude_iptw, 60) - restricted_mean_survival_time(kmf_med_adt_latitude_iptw, 60),
     'diff_rmst_95': latitude_med_rmst_mos_95.difference_rmst_95,
     'rcount': latitude.query('risk_score < @high_cutoff_latitude and risk_score > @low_cutoff_latitude').shape[0]},
    
    {'trial_name': 'LATITUDE', 
     'risk_group': 'high', 
     'r_trt_mos': abi_latitude_median_os[2],
     'r_trt_mos_95': latitude_high_rmst_mos_95.mos_A_95,
     'r_cont_mos': adt_latitude_median_os[2],
     'r_cont_mos_95': latitude_high_rmst_mos_95.mos_B_95,
     'r_mos_diff': abi_latitude_median_os[2] - adt_latitude_median_os[2], 
     'rct_trt_arm': 53.3,
     'rct_cont_arm': 36.5,
     'rct_mos_diff': 53.3-36.5,
     'trt_rmst': restricted_mean_survival_time(kmf_high_abi_latitude_iptw, 60),
     'trt_rmst_95': latitude_high_rmst_mos_95.rmst_A_95,
     'cont_rmst': restricted_mean_survival_time(kmf_high_adt_latitude_iptw, 60),
     'cont_rmst_95': latitude_high_rmst_mos_95.rmst_B_95,
     'diff_rmst': restricted_mean_survival_time(kmf_high_abi_latitude_iptw, 60) - restricted_mean_survival_time(kmf_high_adt_latitude_iptw, 60),
     'diff_rmst_95': latitude_high_rmst_mos_95.difference_rmst_95,
     'rcount': latitude.query('risk_score >= @high_cutoff_latitude').shape[0]},
    
    {'trial_name': 'LATITUDE', 
     'risk_group': 'all', 
     'r_hr': latitude_hr_all.hazard_ratios_['adt_abi'],
     'r_hr_95': [latitude_hr_all.summary.loc['adt_abi']['exp(coef) lower 95%'], latitude_hr_all.summary.loc['adt_abi']['exp(coef) upper 95%']],
     'r_trt_mos': abi_latitude_median_os[3],
     'r_trt_mos_95': latitude_all_rmst_mos_95.mos_A_95,
     'r_cont_mos': adt_latitude_median_os[3],
     'r_cont_mos_95': latitude_all_rmst_mos_95.mos_B_95,
     'r_mos_diff': abi_latitude_median_os[3] - adt_latitude_median_os[3], 
     'rct_trt_arm': 53.3,
     'rct_cont_arm': 36.5,
     'rct_mos_diff': 53.3-36.5,
     'rcount': latitude.shape[0]}
]

## Part 3. Combining dictionaries 

In [203]:
data_combined = chaarted_data + latitude_data

In [204]:
rtrials_mos_rmst_boot = pd.DataFrame(data_combined)

In [205]:
rtrials_mos_rmst_boot

,trial_name,risk_group,r_trt_mos,r_trt_mos_95,r_cont_mos,r_cont_mos_95,r_mos_diff,rct_trt_arm,rct_cont_arm,rct_mos_diff,trt_rmst,trt_rmst_95,cont_rmst,cont_rmst_95,diff_rmst,diff_rmst_95,rcount,rcount_chemo,r_hr,r_hr_95
0,CHAARTED,low,65.533333,"[52.06666666666667, nan]",67.766667,"[62.9, 72.50333333333333]",-2.233333,57.6,44.0,13.6,48.762483,"[46.36637430409277, 51.44123648669106]",47.947373,"[47.01367458370921, 48.841637573746894]",0.815110,"[-1.7133924668499652, 3.7823850723352876]",2466,439.0,NaN,NaN
1,CHAARTED,medium,53.000000,"[47.3, 60.56666666666667]",40.700000,"[38.8975, 43.266666666666666]",12.300000,57.6,44.0,13.6,45.509112,"[43.02830426821335, 47.84254625097214]",39.360818,"[38.48587947217297, 40.243256620667594]",6.148293,"[3.5359889762915544, 8.595687442455036]",2465,424.0,NaN,NaN
2,CHAARTED,high,33.433333,"[27.698333333333334, 34.96666666666667]",21.700000,"[20.5, 23.0]",11.733333,57.6,44.0,13.6,32.323328,"[29.232553786560118, 35.328405087399624]",25.673721,"[24.823881669633725, 26.506346392947805]",6.649607,"[3.4589787840440085, 9.57552550837878]",2466,316.0,NaN,NaN
3,CHAARTED,all,47.300000,"[43.06666666666667, 52.833333333333336]",38.366667,"[37.4, 39.83416666666667]",8.933333,57.6,44.0,13.6,NaN,NaN,NaN,NaN,NaN,NaN,7397,1179.0,0.695578,"[0.6165419767291266, 0.7847457342339457]"
4,LATITUDE,low,inf,"[61.2, nan]",68.166667,"[63.46666666666667, 73.76666666666667]",inf,53.3,36.5,16.8,49.873130,"[47.08938131560391, 52.35561568220965]",48.129469,"[47.33499280246989, 48.92837098077076]",1.743661,"[-1.160991118709693, 4.2817279885513955]",2348,NaN,NaN,NaN
5,LATITUDE,medium,52.566667,"[36.666666666666664, nan]",40.666667,"[38.666666666666664, 42.568333333333335]",11.900000,53.3,36.5,16.8,42.635478,"[38.54489686197694, 46.77611749161571]",39.215615,"[38.278809098462595, 40.15652494996081]",3.419863,"[-0.7599624274018226, 7.817934220689069]",2347,NaN,NaN,NaN
6,LATITUDE,high,25.366667,"[20.966666666666665, 33.266666666666666]",21.366667,"[20.3, 22.866666666666667]",4.000000,53.3,36.5,16.8,28.309201,"[24.60896665308205, 32.98613443035465]",25.547877,"[24.66364833292573, 26.419947100705937]",2.761324,"[-1.2383260582249649, 7.321866510195422]",2348,NaN,NaN,NaN
7,LATITUDE,all,47.100000,"[38.99666666666667, 53.5]",38.466667,"[37.4325, 39.80083333333333]",8.633333,53.3,36.5,16.8,NaN,NaN,NaN,NaN,NaN,NaN,7043,NaN,0.834828,"[0.717542469696957, 0.9712838181512845]"


In [206]:
rtrials_mos_rmst_boot.query('risk_group == "all"')[['trial_name', 'r_hr', 'r_hr_95']]

,trial_name,r_hr,r_hr_95
3,CHAARTED,0.695578,"[0.6165419767291266, 0.7847457342339457]"
7,LATITUDE,0.834828,"[0.717542469696957, 0.9712838181512845]"


In [207]:
rtrials_mos_rmst_boot.query('risk_group != "all"')[['trial_name', 'risk_group', 'diff_rmst', 'diff_rmst_95']]

,trial_name,risk_group,diff_rmst,diff_rmst_95
0,CHAARTED,low,0.815110,"[-1.7133924668499652, 3.7823850723352876]"
1,CHAARTED,medium,6.148293,"[3.5359889762915544, 8.595687442455036]"
2,CHAARTED,high,6.649607,"[3.4589787840440085, 9.57552550837878]"
4,LATITUDE,low,1.743661,"[-1.160991118709693, 4.2817279885513955]"
5,LATITUDE,medium,3.419863,"[-0.7599624274018226, 7.817934220689069]"
6,LATITUDE,high,2.761324,"[-1.2383260582249649, 7.321866510195422]"


In [208]:
rtrials_mos_rmst_boot.to_csv('rtrials_mos_rmst_boot.csv', index = False)